In [3]:
# knjiznice
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import numpy as np
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
import matplotlib.pyplot as plt

In [19]:
# spremenljivke za testiranje
st_izbranih_znacilnic = 5
st_zaporednih = 504
st_napovedi = 1

In [13]:
# branje podatkov iz datoteke
df = pd.read_excel('iir_podatkovna_zbirka.xlsx')
df

,Unnamed: 0,time,global energy,diffusive energy,mean T,min T,max T,T,precipitation,mean pressure,min pressure,max pressure,mean rel. hum.,min rel. hum.,max rel. hum.,rel. hum.,wind speed,wind direction,max gust
0,0,2019-03-20 19:30:00,0.0,0.0,3.1,3.0,3.3,3.2,0.0,953.0,953.0,953.0,50.0,49.0,51.0,50.0,2.8,40.0,5.1
1,1,2022-04-16 15:50:00,NaN,NaN,9.4,9.2,9.6,9.2,NaN,942.0,942.0,943.0,63.0,61.0,66.0,62.0,3.1,29.0,6.3
2,2,2017-01-29 02:00:00,0.0,0.0,-7.4,-8.1,-7.0,-7.1,0.0,944.0,944.0,944.0,94.0,92.0,97.0,95.0,1.0,190.0,1.5
3,3,2021-10-09 07:20:00,NaN,NaN,5.4,5.3,5.6,5.6,NaN,946.0,946.0,947.0,81.0,80.0,83.0,80.0,5.4,47.0,8.6
4,4,2019-11-15 06:20:00,NaN,NaN,3.9,3.9,4.0,3.9,NaN,931.0,931.0,931.0,98.0,97.0,99.0,98.0,2.7,11.0,3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311035,311035,2022-09-13 22:00:00,0.0,0.0,16.3,16.2,16.4,16.2,0.0,936.0,936.0,936.0,86.0,85.0,86.0,86.0,4.6,201.0,7.2
311036,311036,2019-09-20 17:40:00,NaN,NaN,11.3,11.3,11.4,11.3,NaN,948.0,948.0,948.0,59.0,57.0,61.0,59.0,1.5,56.0,2.5
311037,311037,2022-08-28 12:30:00,528.0,299.0,23.0,22.8,23.2,22.9,0.0,938.0,938.0,938.0,61.0,56.0,66.0,63.0,2.0,193.0,3.3
311038,311038,2021-03-06 22:40:00,NaN,NaN,-1.6,-1.7,-1.5,-1.7,NaN,950.0,950.0,950.0,48.0,48.0,48.0,48.0,0.6,320.0,1.0


In [14]:
# pretvorimo time v datetime, sort by time da imamo zaporedje in drop time da ne ovira
df['time'] = pd.to_datetime(df['time'])
df.sort_values(by='time', inplace=True)
df = df.drop(['time'], axis=1)
df

,Unnamed: 0,global energy,diffusive energy,mean T,min T,max T,T,precipitation,mean pressure,min pressure,max pressure,mean rel. hum.,min rel. hum.,max rel. hum.,rel. hum.,wind speed,wind direction,max gust
288911,288911,0.0,0.0,-0.6,-0.7,-0.5,-0.7,0.0,948.0,948.0,948.0,34.0,34.0,35.0,35.0,2.6,6.0,2.9
125241,125241,NaN,NaN,-0.7,-0.8,-0.6,-0.8,NaN,948.0,948.0,948.0,35.0,34.0,35.0,35.0,2.6,8.0,2.9
15657,15657,NaN,NaN,-1.0,-1.0,-0.8,-1.0,NaN,948.0,948.0,948.0,36.0,35.0,36.0,36.0,2.4,14.0,2.7
66656,66656,0.0,0.0,-0.8,-1.0,-0.7,-0.7,0.0,948.0,948.0,948.0,35.0,35.0,36.0,35.0,2.0,24.0,2.4
182713,182713,NaN,NaN,-0.7,-0.7,-0.6,-0.7,NaN,948.0,948.0,948.0,34.0,33.0,35.0,33.0,1.9,23.0,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3154,3154,NaN,NaN,-0.1,-0.1,-0.1,-0.1,NaN,941.0,941.0,941.0,97.0,96.0,98.0,97.0,5.3,29.0,7.8
109931,109931,NaN,NaN,-0.1,-0.2,-0.1,-0.2,NaN,941.0,941.0,941.0,97.0,97.0,98.0,97.0,5.1,25.0,7.6
31904,31904,0.0,0.0,-0.2,-0.2,-0.2,-0.2,0.2,941.0,941.0,941.0,97.0,96.0,98.0,97.0,5.7,29.0,8.0
154759,154759,NaN,NaN,-0.2,-0.2,-0.2,-0.2,NaN,941.0,941.0,941.0,97.0,96.0,98.0,97.0,5.1,29.0,7.8


In [15]:
# zapolnimo missing data 
imp = IterativeImputer()
imp.fit(df)
temp_df = imp.transform(df)
df = pd.DataFrame(temp_df, columns=df.columns)
df

,Unnamed: 0,global energy,diffusive energy,mean T,min T,max T,T,precipitation,mean pressure,min pressure,max pressure,mean rel. hum.,min rel. hum.,max rel. hum.,rel. hum.,wind speed,wind direction,max gust
0,288911.0,0.000000,0.000000,-0.6,-0.7,-0.5,-0.7,0.000000,948.0,948.0,948.0,34.0,34.0,35.0,35.0,2.6,6.0,2.9
1,125241.0,47.814247,-3.283533,-0.7,-0.8,-0.6,-0.8,-0.231041,948.0,948.0,948.0,35.0,34.0,35.0,35.0,2.6,8.0,2.9
2,15657.0,53.940140,0.228771,-1.0,-1.0,-0.8,-1.0,-0.214562,948.0,948.0,948.0,36.0,35.0,36.0,36.0,2.4,14.0,2.7
3,66656.0,0.000000,0.000000,-0.8,-1.0,-0.7,-0.7,0.000000,948.0,948.0,948.0,35.0,35.0,36.0,35.0,2.0,24.0,2.4
4,182713.0,113.510726,13.932471,-0.7,-0.7,-0.6,-0.7,-0.235367,948.0,948.0,948.0,34.0,33.0,35.0,33.0,1.9,23.0,2.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311035,3154.0,-57.196301,6.752438,-0.1,-0.1,-0.1,-0.1,0.104461,941.0,941.0,941.0,97.0,96.0,98.0,97.0,5.3,29.0,7.8
311036,109931.0,-71.929606,0.029726,-0.1,-0.2,-0.1,-0.2,0.116828,941.0,941.0,941.0,97.0,97.0,98.0,97.0,5.1,25.0,7.6
311037,31904.0,0.000000,0.000000,-0.2,-0.2,-0.2,-0.2,0.200000,941.0,941.0,941.0,97.0,96.0,98.0,97.0,5.7,29.0,8.0
311038,154759.0,-59.744355,7.282170,-0.2,-0.2,-0.2,-0.2,0.121172,941.0,941.0,941.0,97.0,96.0,98.0,97.0,5.1,29.0,7.8


In [16]:
# locimo na podatke in vrednosti za napoved
X = df.drop(['global energy'], axis=1)
Y = df['global energy'].to_numpy()

In [17]:
# izracunamo katere znacilnice vzeti
mutual_info = mutual_info_regression(X, Y)
izbrane = X.columns[np.argsort(mutual_info)[-st_izbranih_znacilnic:]]
X = X[izbrane]
print(izbrane)

Index(['mean rel. hum.', 'rel. hum.', 'min rel. hum.', 'precipitation',
       'diffusive energy'],
      dtype='object')


In [20]:
# split data na train in test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=4032, shuffle=False)

In [21]:
Y_train.shape[1]

IndexError: tuple index out of range

In [ ]:
# standardiziranje podatkov
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# pripravimo timeseries

dolzina_X_train = len(X_train)
dolzina_X_test = len(X_test)

temp_Xtr = []
temp_Ytr = []

for i in range(st_zaporednih, dolzina_X_train - st_napovedi + 1):
    temp_Xtr.append(X_train[i - st_zaporednih:i, :])
    temp_Ytr.append(Y_train[i + st_napovedi - 1:i + st_napovedi])

X_train = np.array(temp_Xtr)
Y_train = np.array(temp_Ytr)

temp_Xte = []
temp_Yte = []

for i in range(st_zaporednih, dolzina_X_test - st_napovedi + 1):
    temp_Xte.append(X_test[i - st_zaporednih:i, :])
    temp_Yte.append(Y_test[i + st_napovedi - 1:i + st_napovedi])

X_test = np.array(temp_Xte)
Y_test = np.array(temp_Yte)

In [ ]:
# pripravimo model
model = Sequential()
model.add(LSTM(128, input_shape=(st_zaporednih, st_izbranih_znacilnic), return_sequences=True))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1]))

cp = ModelCheckpoint('model/', save_best_only=True)
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
model.fit(X_train, Y_train, epochs=20, steps_per_epoch=500, validation_data=(X_test, Y_test), batch_size=64,callbacks=[cp])

model = load_model('model/')

model.save('model.h5')

In [ ]:
predictions2 = model2.predict(X_test)
result2 = model2.evaluate(X_test, Y_test, verbose=0)
print(f"Test loss: {result2}")

mae2 = mean_absolute_error(Y_test, predictions2)
print(f"Mean Absolute Error: {mae2}")

mse2 = mean_squared_error(Y_test, predictions2)
print(f"Mean Squared Error: {mse2}")

evs2 = explained_variance_score(Y_test, predictions2)
print(f"Explained Variance Score: {evs2}")

plt.plot(Y_test, label='True values')
plt.plot(predictions2, label='Predictions')
plt.legend()
plt.show()